In [1]:
import sys
sys.path.append('/home/jovyan/')

from api.convertsigma import ConvertSigma
from api.spark import Spark
from api.webhook import Webhook

SIGMAHELPER = ConvertSigma()
SPARKHELPER = Spark()
WEBHOOKHELPER = Webhook()

In [2]:
# Download and Convert Sigma rules from S3 bucket
SIGMAHELPER.main()

[+] Downloaded rules/osquery_potential_gatekeeper_bypass.yaml to sigma/rules/osquery/osquery_potential_gatekeeper_bypass.yaml
[+] Downloaded rules/osquery_suspicious_osascript.yaml to sigma/rules/osquery/osquery_suspicious_osascript.yaml
[+] Processing Sigma input osquery_potential_gatekeeper_bypass.yaml
[+] Processing Sigma input osquery_suspicious_osascript.yaml
[*] Completed - rules in sigma/rules/output


In [3]:
# Initialise SparkSession
spark = SPARKHELPER.load('alerting', '2')

```
✗ kubectl get pods
NAME                                  READY   STATUS    RESTARTS   AGE
alerting-1589542277635-exec-1         1/1     Running   0          2m2s
alerting-1589542278121-exec-2         1/1     Running   0          2m1s
```

In [4]:
# Read Logs from S3 Bucket using Spark
s3path = "s3a://your-own-bucket/sample-logs/*"
df = spark.read.json(s3path)

In [5]:
# Nested DataFrame Schema
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- calendarTime: string (nullable = true)
 |-- columns: struct (nullable = true)
 |    |-- cmdline: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- on_disk: string (nullable = true)
 |    |-- parent: string (nullable = true)
 |    |-- parentpath: string (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- pid: string (nullable = true)
 |    |-- ppid: string (nullable = true)
 |    |-- sha256: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- uid: string (nullable = true)
 |    |-- username: string (nullable = true)
 |-- counter: long (nullable = true)
 |-- decorations: struct (nullable = true)
 |    |-- host_uuid: string (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- osquery_version: string (nullable = true)
 |-- epoch: long (nullable = true)
 |-- hostIdentifier: string (nullable = true)
 |-- name: string (nullable = true)
 |-- unixTime: long (nu

In [6]:
# Flatten the DataFrame
flat_df = SPARKHELPER.flatten(df)

In [7]:
# Flattened Schema
flat_df.printSchema()

root
 |-- action: string (nullable = true)
 |-- calendarTime: string (nullable = true)
 |-- counter: long (nullable = true)
 |-- epoch: long (nullable = true)
 |-- hostIdentifier: string (nullable = true)
 |-- name: string (nullable = true)
 |-- unixTime: long (nullable = true)
 |-- columns_cmdline: string (nullable = true)
 |-- columns_name: string (nullable = true)
 |-- columns_on_disk: string (nullable = true)
 |-- columns_parent: string (nullable = true)
 |-- columns_parentpath: string (nullable = true)
 |-- columns_path: string (nullable = true)
 |-- columns_pid: string (nullable = true)
 |-- columns_ppid: string (nullable = true)
 |-- columns_sha256: string (nullable = true)
 |-- columns_state: string (nullable = true)
 |-- columns_uid: string (nullable = true)
 |-- columns_username: string (nullable = true)
 |-- decorations_host_uuid: string (nullable = true)
 |-- decorations_hostname: string (nullable = true)
 |-- decorations_osquery_version: string (nullable = true)



In [8]:
# Register SQL table with the DF
flat_df.registerTempTable("tempDF")

In [9]:
# Example data from DF
spark.sql("""
SELECT decorations_hostname, columns_name, columns_parent FROM tempDF
LIMIT 5
""").show(truncate=False)

+--------------------+-----------------------------------------------+--------------+
|decorations_hostname|columns_name                                   |columns_parent|
+--------------------+-----------------------------------------------+--------------+
|JAYDENHOST          |AddressBookSourceSync                          |launchd       |
|JAYDENHOST          |com.apple.CloudDocs.MobileDocumentsFileProvider|launchd       |
|JAYDENHOST          |wifiFirmwareLoader                             |launchd       |
|JAYDENHOST          |MRT                                            |launchd       |
|JAYDENHOST          |keybagd                                        |launchd       |
+--------------------+-----------------------------------------------+--------------+



In [10]:
# Detecting AppleScript and Gatekeeper abuse
spark.sql("""
SELECT decorations_hostname, name, columns_name, columns_cmdline, columns_parent FROM tempDF
WHERE columns_cmdline LIKE '%osascript%' OR columns_cmdline LIKE '%xattr%'
""").show(truncate=False)

+--------------------+------------------+------------+----------------------------------------------------+--------------+
|decorations_hostname|name              |columns_name|columns_cmdline                                     |columns_parent|
+--------------------+------------------+------------+----------------------------------------------------+--------------+
|JAYDENHOST          |pack/mac/processes|osascript   |osascript -e display dialog "hello world"           |zsh           |
|JAYDENHOST          |pack/mac/processes|xattr       |xattr -r -d com.apple.quarantine /tmp/helloworld.app|zsh           |
+--------------------+------------------+------------+----------------------------------------------------+--------------+



In [11]:
!cat sigma/rules/osquery/osquery_potential_gatekeeper_bypass.yaml

title: osquery_potential_gatekeeper_bypass
description: Detect commands used to bypass gatekeeper.
references:
    - https://attack.mitre.org/techniques/T1144/
author: Jayden Zheng
logsource:
    product: osquery
    service: osquery
tags:
    - MITRE.T1144
detection:
    selection:
        name: "pack/mac/processes"
        columns_name: 'xattr'
        columns_cmdline: '*com.apple.quarantine*'
    condition: selection
level: high


In [12]:
!sigma/tools/sigmac -c sigma/config/osquery.yml -t sql sigma/rules/osquery/osquery_potential_gatekeeper_bypass.yaml

(sourcetype = "osquery" AND name = "pack/mac/processes" AND columns_name = "xattr" AND columns_cmdline LIKE "%com.apple.quarantine%")


In [13]:
spark.sql("""
SELECT decorations_hostname, name, columns_name, columns_cmdline, columns_parent FROM tempDF
WHERE name = "pack/mac/processes" AND columns_name = "xattr" AND columns_cmdline LIKE "%com.apple.quarantine%"
""").show(truncate=False)

+--------------------+------------------+------------+----------------------------------------------------+--------------+
|decorations_hostname|name              |columns_name|columns_cmdline                                     |columns_parent|
+--------------------+------------------+------------+----------------------------------------------------+--------------+
|JAYDENHOST          |pack/mac/processes|xattr       |xattr -r -d com.apple.quarantine /tmp/helloworld.app|zsh           |
+--------------------+------------------+------------+----------------------------------------------------+--------------+



In [14]:
!cat sigma/rules/osquery/osquery_suspicious_osascript.yaml

title: osquery_suspicious_osascript
description: Detect commands used to execute AppleScript through osascript.
references:
    - https://attack.mitre.org/techniques/T1155/
author: Jayden Zheng
logsource:
    product: osquery
    service: osquery
tags:
    - MITRE.T1155
detection:
    selection:
        name: "pack/mac/processes"
        columns_name: 'osascript'
        columns_cmdline: '*-e*'
    condition: selection
level: high


In [15]:
!sigma/tools/sigmac -c sigma/config/osquery.yml -t sql sigma/rules/osquery/osquery_suspicious_osascript.yaml

(sourcetype = "osquery" AND name = "pack/mac/processes" AND columns_name = "osascript" AND columns_cmdline LIKE "%-e%")


In [16]:
spark.sql("""
SELECT decorations_hostname, name, columns_name, columns_cmdline, columns_parent FROM tempDF
WHERE name = "pack/mac/processes" AND columns_name = "osascript" AND columns_cmdline LIKE "%-e%"
""").show(truncate=False)

+--------------------+------------------+------------+-----------------------------------------+--------------+
|decorations_hostname|name              |columns_name|columns_cmdline                          |columns_parent|
+--------------------+------------------+------------+-----------------------------------------+--------------+
|JAYDENHOST          |pack/mac/processes|osascript   |osascript -e display dialog "hello world"|zsh           |
+--------------------+------------------+------------+-----------------------------------------+--------------+



In [21]:
!cat sigma/rules/output/osquery.rules

osquery_potential_gatekeeper_bypass && 3 && name = "pack/mac/processes" AND columns_name = "xattr" AND columns_cmdline LIKE "%com.apple.quarantine%"
osquery_suspicious_osascript && 3 && name = "pack/mac/processes" AND columns_name = "osascript" AND columns_cmdline LIKE "%-e%"


In [17]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
def explore(df):
    output = "%s" % (df)
    WEBHOOKHELPER.alert(title, output, level, 'osquery')

In [19]:
for line in open('sigma/rules/output/osquery.rules', 'r'):
    try:
        # Retrieve the title, priority and rule
        splitline = line.split('&&')
        title = splitline[0].rstrip()
        priority = splitline[1].lstrip().rstrip()
        rule = "%s" % (splitline[2].lstrip().rstrip())
        if "1" in priority:
            level = "Low"
        elif "2" in priority:
            level = "Medium"
        elif "3" in priority:
            level = "High"
        else:
            level = "Unknown"

        # Append the rule to the SQL statement after WHERE
        filtertorun = 'SELECT * FROM tempDF WHERE %s' % (rule)

        # Processing and Alert
        result = spark.sql(filtertorun)
        countevent = "%s" % (result.count())
        print("[+] Processed: %s - %s event(s)" % (title, countevent))
        if countevent != '0':
            pandas_df = result.toPandas()
            pandas_df = pandas_df.dropna(axis=1, how='all')
            pandas_df.apply(explore, axis=1)
    except:
        print("[*] Error")

[+] Processed: osquery_potential_gatekeeper_bypass - 1 event(s)
[+] Processed: osquery_suspicious_osascript - 1 event(s)


In [20]:
spark.stop()